## Reparse tmf of zenodo dataset

In [1]:
from starling_rhythm.mfdfa import tmf
from starling_rhythm.utils.paths import PROCESSED_DIR
from starling_rhythm.utils.paths import DATA_DIR

/mnt/cube/j8xing/conda_envs/txori/starling_rhythm/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from scipy.io.wavfile import read

In [3]:
from pathlib2 import Path
from glob import glob

## Find all the wavs

In [4]:
import pandas as pd
import numpy as np

In [5]:
wav_paths = list(Path(DATA_DIR).rglob("20*.wav"))

In [6]:
wavs = pd.DataFrame(
    {
        'wavs': wav_paths
    }
)

In [7]:
bID = []
for index, row in wavs.iterrows():
    bID.append(str(row['wavs']).split('/')[-3])
    
wavs['bID'] = bID

In [8]:
len(wavs)

10499

In [9]:
wavs.head()

,wavs,bID
0,/mnt/cube/Datasets/public_ds_starlings_ts_2019...,b1053
1,/mnt/cube/Datasets/public_ds_starlings_ts_2019...,b1053
2,/mnt/cube/Datasets/public_ds_starlings_ts_2019...,b1053
3,/mnt/cube/Datasets/public_ds_starlings_ts_2019...,b1053
4,/mnt/cube/Datasets/public_ds_starlings_ts_2019...,b1053


In [10]:
np.unique(wavs.bID)

array(['B335', 'B336', 'B337', 'B338', 'b1053', 'b1060', 'b1066', 'b1076',
       'b1077', 'b1125', 'b1143', 'b1145', 'b1159', 'b1160'], dtype=object)

## Goals
Calculate white noise mean  
Calculate empirical tmf, get their p values  

In [11]:
from joblib import Parallel, delayed
from tqdm.autonotebook import tqdm

In [12]:
import psutil
psutil.Process().nice(19)# if on *ux

In [13]:
n_jobs = 42
verbose = 0

In [14]:
with Parallel(n_jobs = n_jobs, verbose = verbose) as parallel:
    tmfs = parallel(
        delayed(tmf)(
            address,
            ns = 32,
            amp_env_sr = 200,
            return_p = True
        )
        for address in tqdm(wavs['wavs'], desc = 'getting all the real ones')
    )

getting all the real ones:   1%|          | 126/10499 [04:26<7:36:03,  2.64s/it]/mnt/cube/j8xing/conda_envs/txori/starling_rhythm/lib/python3.10/site-packages/joblib/externals/loky/process_executor.py:702: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
getting all the real ones: 100%|██████████| 10499/10499 [13:15:19<00:00,  4.55s/it] 


In [15]:
tmfs, ps = zip(*tmfs)

In [16]:
wavs['tmf'] = tmfs
wavs['ps'] = ps

In [17]:
from starling_rhythm.utils.paths import PROCESSED_DIR, ensure_dir
SAVE_PATH = PROCESSED_DIR / 'starling_zenodo_2019_revised.pickle'
ensure_dir(SAVE_PATH)

In [18]:
wavs.to_pickle(SAVE_PATH)